# Assignment 10: AWS SageMaker Tutorial

This tutorial, reproduced from [here](https://aws.amazon.com/getting-started/hands-on/build-train-deploy-machine-learning-model-sagemaker/), demonstrates how to use AWS SageMaker to train an XGBoost classifier on [direct marketing campaign data](https://archive.ics.uci.edu/ml/datasets/bank+marketing) to predict whether client will subscribe a term deposit.

The assignment is comprised of the following sections:
* <a href="#import">**Import libraries**</a>
* <a href="#make-bucket">**Create S3 bucket**</a>
* <a href="#download">**Download and split train and test data**</a>
* <a href="#train">**Train the XGBoost classifier**</a>
* <a href="#deploy">**Deploy the trained model**</a>
* <a href="#test">**Predict on test data**</a>
* <a href="#analyze">**Analyze outputs**</a>
* <a href="#clean">**Clean up**</a>

## <a name="import">Import libraries</a>
* In SageMaker we don't have to install the libraries because the `conda_python3` environment already includes them.

In [1]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer
from sagemaker.serializers import CSVSerializer

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the us-west-2 region. You will use the 433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [2]:
bucket_name = 'yogotha-learns' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    else: 
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


## <a name="download">Download and spit test and train data</a>

In [3]:
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

model_data.head(3)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,37,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0


In [4]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


## <a name="train">Train the ML model</a>

In [5]:
# Save train data locally
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)

# Upload locally saved train data to S3 bucket
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

# Load the data from S3 into a sagemaker.inputs.TrainingInput object
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [6]:
# Instantiate a SageMaker session and XGBoost estimator
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(xgboost_container,role, instance_count=1, instance_type='ml.m4.xlarge',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)
# Update XGBoost estimator with desired hyperparameters
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)
# Fit the model
xgb.fit({'train': s3_input_train})

INFO:sagemaker:Creating training-job with name: xgboost-2023-04-09-14-51-12-916


2023-04-09 14:51:13 Starting - Starting the training job...
2023-04-09 14:51:38 Starting - Preparing the instances for training.........
2023-04-09 14:52:47 Downloading - Downloading input data...
2023-04-09 14:53:17 Training - Downloading the training image...
2023-04-09 14:54:02 Training - Training image download completed. Training in progress..Arguments: train
[2023-04-09:14:54:14:INFO] Running standalone xgboost training.
[2023-04-09:14:54:14:INFO] Path /opt/ml/input/data/validation does not exist!
[2023-04-09:14:54:14:INFO] File size need to be processed in the node: 3.38mb. Available memory size in the node: 8607.91mb
[2023-04-09:14:54:14:INFO] Determined delimiter of CSV input is ','
[14:54:14] S3DistributionType set as FullyReplicated
[14:54:14] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[14:54:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 14 pruned nodes, max_depth=5
[0]#011tra

## <a name="deploy">Deploy the estimator to SageMaker</a>

In [7]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2023-04-09-14-54-58-470
INFO:sagemaker:Creating endpoint-config with name xgboost-2023-04-09-14-54-58-470
INFO:sagemaker:Creating endpoint with name xgboost-2023-04-09-14-54-58-470


-------!

## <a name="test">Predict on test data using deployed model</a>

In [8]:
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


## <a name="analyze">Analyze the model outputs</a>
* Print classification report on model performance on test data

In [9]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.5%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10769)    37% (167)
Purchase        10% (1133)     63% (288) 



## <a name="clean">Clean up</a>
* Delete the endpoint
* Delete the training artifcats and S3 bucket

In [10]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)

INFO:sagemaker:Deleting endpoint configuration with name: xgboost-2023-04-09-14-54-58-470
INFO:sagemaker:Deleting endpoint with name: xgboost-2023-04-09-14-54-58-470


In [11]:
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'AC2CYGK75MNJBNYK',
   'HostId': 'qfeMHAO2j21ikmymL6xt5HbgazJnXUWvqRIXFgDAHVUmWzY3rpqu5wYA2fECKMepdBBJ1lJpbjc=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'qfeMHAO2j21ikmymL6xt5HbgazJnXUWvqRIXFgDAHVUmWzY3rpqu5wYA2fECKMepdBBJ1lJpbjc=',
    'x-amz-request-id': 'AC2CYGK75MNJBNYK',
    'date': 'Sun, 09 Apr 2023 14:58:33 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2023-04-09-14-51-12-916/profiler-output/system/training_job_end.ts'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/train/train.csv'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2023-04-08-21-29-17-350/profiler-output/system/incremental/2023040821/1680989520.algo-1.json'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2023-04-09-14-51-12-916/debug-output/training_job_end.ts'},
   {'Key': 'sagem